# Neural Network from scratch

In [ ]:
import numpy as np 
import pandas as pd 

csv from https://www.kaggle.com/competitions/digit-recognizer/data :

In [ ]:
data = pd.read_csv(r"urpathto\train.csv")

Convert Data to NumPy Array: The input data is converted into a NumPy array using np.array() function.

Shuffle Data: The rows of the data array are shuffled randomly using the np.random.shuffle() function.

Prepare Development Data: A subset of the shuffled data consisting of the first 1000 rows is extracted and transposed (T). The first element of this subset is assigned to Y_dev, while the remaining elements are assigned to X_dev. The values in X_dev are divided by 255 to normalize them.

Prepare Training Data: The remaining rows of the shuffled data (from index 1000 onwards) are extracted and transposed. The first element of this subset is assigned to Y_train, while the remaining elements are assigned to X_train. The values in X_train are divided by 255 for normalization.

Get Training Data Shape: The _ placeholder variable is used to disregard the first value returned by the X_train.shape method, and the number of columns in X_train is assigned to m_train.

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T 
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255. 
_,m_train = X_train.shape

The init_params() function initializes the parameters (weights and biases) for a neural network.  

We initialize the matrices with random values drawn from a normal distribution.  

Then return the initialized weight and bias matrices (W1, b1, W2, and b2) as the output of the function.  

In [ ]:
def init_params():
    W1 = np.random.normal(size=(10, 784)) * np.sqrt(1./(784))
    b1 = np.random.normal(size=(10, 1)) * np.sqrt(1./10)
    W2 = np.random.normal(size=(10, 10)) * np.sqrt(1./20)
    b2 = np.random.normal(size=(10, 1)) * np.sqrt(1./(784))
    return W1, b1, W2, b2

The ReLU function applies the Rectified Linear Unit (ReLU) activation function element-wise to the input array Z.  

The ReLU activation function is applied element-wise to Z using np.maximum(). This function compares each element of Z with 0 and returns the element itself if it is greater than 0, otherwise, it returns 0.

The result of the element-wise maximum operation is stored in the array A.

Finally, the function returns the output array A after applying the ReLU activation function.

In [ ]:
def ReLU(Z):
    return np.maximum(Z, 0)

The softmax function applies the softmax function element-wise to the input array Z. 

The exponential of each element in Z is computed using np.exp() and stored in the array exp_Z.

The sum of the exponential values in exp_Z is computed using np.sum() and stored in the variable sum_exp_Z.

Each element of the exponential values in exp_Z is divided by the sum sum_exp_Z, resulting in the softmax probabilities. The division is performed element-wise.

The resulting softmax probabilities are stored in the array A.

Finally, the function returns the output array A after applying the softmax function.

In [ ]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

The forward_prop function performs forward propagation for a neural network. It takes the weight matrices (W1 and W2), bias vectors (b1 and b2), and input data (X) as inputs.  

The linear transformation of the input X for the first layer is computed using matrix multiplication (dot) of W1 and X, and adding the bias vector b1. The result is stored in the array Z1.

The ReLU activation function is applied to the first layer output Z1 using the ReLU function. The result is stored in the array A1.

The linear transformation of the first layer output A1 for the second layer is computed using matrix multiplication (dot) of W2 and A1, and adding the bias vector b2. The result is stored in the array Z2.

The softmax activation function is applied to the second layer output Z2 using the softmax function. The result is stored in the array A2.

Finally, the function returns the intermediate values (Z1 and Z2) and the final outputs (A1 and A2) obtained during the forward propagation.

In [ ]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

The one_hot function converts categorical labels into a one-hot encoded representation. 

An array of zeros is created using np.zeros() with the shape (Y.size, Y.max() + 1). This shape ensures that each unique value in Y will have its corresponding column in the one-hot encoded array.

The elements at the indices corresponding to the values in Y are set to 1 in the one_hot_Y array using advanced indexing. This creates the one-hot encoded representation.

The one_hot_Y array is transposed using .T to match the convention where each column represents a one-hot encoded vector.

Finally, the function returns the one-hot encoded array one_hot_Y.

In [ ]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

The deriv_ReLU function computes the derivative of the Rectified Linear Unit (ReLU) activation function.  

The derivative of the ReLU activation function is computed by checking if each element of Z is greater than 0. The comparison operation (>) returns a boolean array where True represents values greater than 0 and False represents values less than or equal to 0.

In [ ]:
def deriv_ReLU(Z):
    return Z > 0

The backward_prop function performs backward propagation to compute gradients for the neural network. It takes the intermediate values (Z1, A1, Z2, A2), weight matrices (W1, W2), input data (X), and true labels (Y) as inputs.  

The one-hot encoded representation of the true labels Y is computed using the one_hot function and stored in the variable one_hot_Y.

The derivative of the second layer output A2 with respect to the loss is computed by subtracting one_hot_Y from A2. The result is stored in the variable dZ2.

The gradients of the second layer weights (dW2) and biases (db2) are computed using the formulas provided in the code.

The derivative of the first layer output A1 with respect to the loss is computed by performing matrix multiplication (dot) of W2.T and dZ2, and element-wise multiplication (*) with the derivative of ReLU (deriv_ReLU(Z1)). The result is stored in the variable dZ1.

The gradients of the first layer weights (dW1) and biases (db1) are computed using the formulas provided in the code.

Finally, the function returns the computed gradients dW1, db1, dW2, and db2.

In [ ]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2   

The update_params function updates the parameters (weights and biases) of the neural network based on the calculated gradients. 

The function is defined with the input arguments W1, b1, W2, b2, dW1, db1, dW2, db2, and alpha.

The first layer weights W1 and biases b1 are updated by subtracting the product of the learning rate alpha and the corresponding gradients dW1 and db1.

The second layer weights W2 and biases b2 are updated by subtracting the product of the learning rate alpha and the corresponding gradients dW2 and db2.

Finally, the function returns the updated parameters W1, b1, W2, and b2.

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

The get_predictions function retrieves the predicted labels based on the output of the neural network. 

The function is defined with A2 as the input argument, which represents the output array of the neural network.

The np.argmax() function is used to find the index of the maximum value along axis 0 (column-wise) in A2. This corresponds to the predicted label for each input.

The resulting indices are stored in the predictions array.

Finally, the function returns the array of predicted labels predictions.

In [ ]:
def get_predictions(A2):
    return np.argmax(A2,0)

The get_accuracy function calculates the accuracy of the predicted labels compared to the true labels.  

The function is defined with predictions and Y as the input arguments, where predictions represents the array of predicted labels and Y represents the true labels.

The print() statement is included to display the predicted labels and true labels, which can be useful for debugging or analysis purposes.

The expression predictions == Y creates a boolean array where True values indicate correct predictions and False values indicate incorrect predictions.

The np.sum() function sums up the number of True values in the boolean array, which represents the number of correct predictions.

The number of correct predictions is divided by the total number of examples (Y.size) to compute the accuracy.

In [ ]:
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

The gradient_descent function performs gradient descent optimization for the neural network. It takes the input data (X), true labels (Y), learning rate (alpha), and number of iterations (iterations) as inputs.  

The function is defined with the input arguments X, Y, alpha, and iterations.

The parameters of the neural network (W1, b1, W2, b2) are initialized using the init_params function.

A loop is executed for the specified number of iterations.

In each iteration, the forward propagation is performed to obtain the intermediate values and outputs of the neural network (Z1, A1, Z2, A2).

The backward propagation is performed to compute the gradients of the parameters (dW1, db1, dW2, db2).

The parameters are updated using the update_params function and gradient descent.

If the iteration is a multiple of 10, the accuracy is computed using the get_accuracy function and printed.

Finally, the function returns the optimized parameters (W1, b1, W2, b2).  

In [ ]:
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)